In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .appName("IngestaoBronze") \
    .getOrCreate()

In [0]:
%run ./Intancia_Containers

In [0]:
# Recuperando Chaves de Acesso
TGT_STORAGE_ACCOUNT = dbutils.secrets.get(scope="secrets-kv", key="tgt-storage-account")
TGT_CONTAINER = dbutils.secrets.get(scope="secrets-kv", key="tgt-container")
TGT_SAS_TOKEN = dbutils.secrets.get(scope="secrets-kv", key="tgt-sas-token")

# Configuring SAS for Lakehouse Container
spark.conf.set(
    f"fs.azure.sas.{TGT_CONTAINER}.{TGT_STORAGE_ACCOUNT}.blob.core.windows.net",
    TGT_SAS_TOKEN
)

# Definindo Path base do LakeHouse
LAKEHOUSE_BASE_PATH = f"wasbs://{TGT_CONTAINER}@{TGT_STORAGE_ACCOUNT}.blob.core.windows.net/bronze_1"

In [0]:
from pyspark.sql.functions import max as spark_max, col

ultima_atualizacao_bronze = (
    spark.table("bronze_1.bronze_control_table")
         .filter(col("container_name") == "BALANCE")
         .select(spark_max("last_ingestion_timestamp").alias("ultima_atualizacao"))
         .collect()[0]["ultima_atualizacao"]
)

print(ultima_atualizacao_bronze)

df_inventario = (
    df_inventario
        .filter(col("container_name") == "BALANCE")
        if ultima_atualizacao_bronze is None
        else df_inventario
            .filter(col("container_name") == "BALANCE")
            .filter(col("modificationTime") > ultima_atualizacao_bronze)
)

display(df_inventario)

In [0]:
from pyspark.sql.functions import col, current_timestamp
from delta.tables import DeltaTable

# iterando nos Arquivos do Container Balance
for row in df_inventario.collect():
    path_origem = row['path']
    container = row['container_name']
    nome_arquivo = row['name']
    
    # Define o caminho de destino (removendo a extensão para o nome da tabela Delta)
    tabela_nome = nome_arquivo.split('.')[0]    
    print(f"⏳ Processando: {tabela_nome}")

    # Verificando se é uma tabela de fato ou dimensão
    tabelas_fact_balance = (
        tabela_nome.startswith("EXP_") or
        tabela_nome.startswith("IMP_")
    )

    # Se for tabela de fato, remove o ano do nome da tabela
    if tabelas_fact_balance:
        tabela_destino = re.sub(r"_\d{4}", "", tabela_nome).lower()
    else:
        tabela_destino = tabela_nome.lower()

    # Definindo o caminho de destino no LakeHouse
    destino_tabela = f"{LAKEHOUSE_BASE_PATH}/{container.lower()}/{tabela_destino}"

    try:
        if nome_arquivo.endswith('.csv'):
            # Lendo arquivo bruto
            df_temp = spark.read.format("csv") \
                .option("header", "true") \
                .option("sep", ";") \
                .option("encoding", "UTF-8") \
                .load(path_origem)
            
            # Convertendo todos os campos para string
            df_temp = df_temp.select(
                [col(c).cast("string").alias(c) for c in df_temp.columns]
            )
            
            if tabelas_fact_balance:
                print(f"✅ Gravando FACT {tabela_nome} (append | partitionBy CO_ANO)")

                df_temp.write \
                    .format("delta") \
                    .mode("append") \
                    .partitionBy("CO_ANO") \
                    .option("mergeSchema", "true") \
                    .save(destino_tabela)
            
            else:
                print(f"♻️ Gravando DIM {tabela_nome} (overwrite)")

                df_temp.write \
                    .format("delta") \
                    .mode("overwrite") \
                    .option("overwriteSchema", "true") \
                    .save(destino_tabela)

            controle_df = spark.createDataFrame(
                [("BALANCE", tabela_destino, path_origem)],
                ["container_name", "table_name", "input_file_name"]
            ).withColumn(
                "last_ingestion_timestamp", current_timestamp()
            )

            controle_df.write \
                .format("delta") \
                .mode("append") \
                .saveAsTable("bronze_1.bronze_control_table")
                
        else:
            print(f"❌ Arquivo {nome_arquivo} no {container} BALANCE não é CSV")

    except Exception as e:
        print(f"❌ Erro ao processar {nome_arquivo}: {e}")